| Name | Type (Shape) | Description |
| --- | --- | --- |
| closed_null_geodesics | list (N,) | list containing $ N $ closed null geodesics
| vortex | list | list containing extracted vortices

In [ ]:
# Import numpy
import numpy as np

# Import shapely library to create polygons and points
from shapely.geometry import Polygon, Point

# tqdm shows progress bar
from tqdm.notebook import tqdm

In [2]:
def _outermost_elliptic_LCS(closed_null_geodesics):
    
    Area = []
    Poly = []
    
    for geodesics in closed_null_geodesics:
        
        if np.sum(geodesics[0]) is not None:
            
            x = geodesics[0]
            y = geodesics[1]         
        
            # compute Polygon object from boundary of closed null geodesics
            Poly.append(Polygon(zip(x, y)))
            
            # compute area of closed null geodesics
            Area.append(Poly[-1].area)

    # if no vortex is found then return 'nan'
    if len(Area) == 0:
        return False
            
    # sort geodesic curves from lowest area to highest area:
    Area_sorted, closed_null_geodesics_sorted = zip(*sorted(zip(Area, closed_null_geodesics)))
    _, Poly_sorted = zip(*sorted(zip(Area, Poly)))
    
    closed_elliptic_LCS = []
    
    for idx, Poly_geodesics in tqdm(enumerate(Poly_sorted), total = len(Poly_sorted)):
        
        coords = np.array(Poly_sorted[idx].exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        # compute centroid of polygon
        x_c = np.mean(x)
        y_c = np.mean(y)
        
        # If first closed elliptic Coherent Structure, then set the center as a reference
        if idx == 0:
            
            closed_elliptic_LCS.append([x_c, y_c, Poly_geodesics])
        
        # Set boolean to create new vortex
        Bool_create_new_closed_elliptic_LCS = True
        
        # iterate over all elliptic LCS
        for v in range(len(closed_elliptic_LCS)):
        
            # If vortex center is contained inside the closed null geodesics --> Bool_create_new_vortex = False
            # --> no new vortex is create
            if Poly_geodesics.contains(Point(closed_elliptic_LCS[v][0], closed_elliptic_LCS[v][1])):
                
                closed_elliptic_LCS[v] = [x_c, y_c, Poly_geodesics]
                
                Bool_create_new_closed_elliptic_LCS = False
                
        # If vortex center is not contained inside the closed null geodesics, then a new vortex is create    
        if Bool_create_new_closed_elliptic_LCS:
            
            closed_elliptic_LCS.append([x_c, y_c, Poly_geodesics])        
    
    # Create list storing outermost elliptic LCS
    outermost_closed_elliptic_LCS = []
    
    # Iteratere over all closed outermost elliptic LCS and append them to 'outermost_closed_elliptic_LCS'
    for v in closed_elliptic_LCS:
        
        coords = np.array(v[2].exterior.coords)
        
        x = coords[:, 0]
        y = coords[:, 1]
        
        outermost_closed_elliptic_LCS.append([x, y])
        
    return outermost_closed_elliptic_LCS